# Frequency reocmmneder

This file contains a system that recommends songs based on their co-occurrence.

This can be run any time after json_to_dataframe.ipynb

In [2]:
import pandas as pd
from tqdm import tqdm
import json

In [2]:
playlist_info = pd.read_hdf('df_data/df_playlists_info.hdf')
playlists = pd.read_hdf('df_data/df_playlists.hdf')
playlist_groups = playlists.groupby('pid')
tracks = pd.read_hdf('df_data/df_tracks.hdf')

# create track occurrences dictionary
track_occurrences = {}
for pid, group in tqdm(playlist_groups):
    playlist_tracks = group['tid'].values

    # create a frequnecy table for each innner track
    for tid in playlist_tracks:
        if tid not in track_occurrences:
            track_occurrences[tid] = {}
            for track in playlist_tracks:
                if track == tid:
                    continue

                track_occurrences[tid][track] = 1
        else:
            for track in playlist_tracks:
                if track == tid:
                    continue

                if track not in track_occurrences[tid]:
                    track_occurrences[tid][track] = 1
                else:
                    track_occurrences[tid][track] += 1

100%|██████████| 14000/14000 [00:39<00:00, 357.06it/s]


In [ ]:
def load_challenge_set():
    f = open('challenge_set.json')
    js = f.read()
    f.close()
    challenge_set = json.loads(js)
    for playlist in challenge_set['playlists']:
        playlist['tracks'] = list(map(lambda x: x['track_uri'], playlist['tracks']))
    return challenge_set

challenge_set = load_challenge_set()

test_playlists = pd.read_hdf('df_data/df_playlists_test.hdf').groupby('pid')
test_playlists_info = pd.read_hdf('df_data/df_playlists_test_info.hdf')
track_frequency = playlists.groupby('tid').size().sort_values(ascending=False)


fp = open('frequency_submission.hdf', 'w')
for i, playlist in tqdm(enumerate(challenge_set['playlists'])):
    playlist_tracks = playlist['tracks']

    # playlist empty, this is bad but just recommend top 500 songs
    if playlist['num_samples'] == 0:
        top_tids = track_frequency.head(500).index
        top_uris = []
        for tid in top_tids:
            track_uri = tracks[tracks['tid'] == tid]['track_uri'].values[0]
            top_uris.append(track_uri)
        fp.write(str(i) +  ', '.join(top_uris) + '\n')
        continue

    # get the track occurrences for the playlist
    track_occurrences_playlist = {}
    for track in playlist_tracks:
        try:
            tid = tracks[tracks['track_uri'] == track]['tid'].values[0]
        except:
            continue

        if tid in track_occurrences:
            for inner_tid, count in track_occurrences[tid].items():
                if inner_tid not in track_occurrences_playlist:
                    track_occurrences_playlist[inner_tid] = count
                else:
                    track_occurrences_playlist[inner_tid] += count
    
    items = list(track_occurrences_playlist.items())
    # sort the track occurrences by count
    sorted_occurrences = sorted(items[:500], key=lambda x: x[1], reverse=True)

    top_tids = list(map(lambda x: x[0], sorted_occurrences))
    top_uris = []
    for tid in top_tids:
        track_uri = tracks[tracks['tid'] == tid]['track_uri'].values[0]
        top_uris.append(track_uri)
    
    fp.write(str(i) + ', ' +  ', '.join(top_uris[:500]) + '\n')

fp.close()


1000it [05:48,  2.87it/s]
